In [2]:
from shapely.geometry import box
import geopandas as gpd

def bbox_gdf(bbox):
    minx, miny, maxx, maxy = bbox
    return gpd.GeoDataFrame(
        geometry=[box(minx, miny, maxx, maxy)],
        crs="EPSG:4326"
    )


In [3]:
# pip install earthaccess geopandas shapely fiona rioxarray xarray matplotlib
from pathlib import Path
import warnings, zipfile, io
import earthaccess as ea
import geopandas as gpd
import xarray as xr
import rioxarray  # registers .rio
import numpy as np
import matplotlib.pyplot as plt

# ---------------- Config ----------------
OKAV_BBOX = (21.0, -21.5, 24.8, -17.5)   # (minLon, minLat, maxLon, maxLat)
START, END = "2024-01-01", "2025-12-31"
OUT = Path("swot_okavango_sample"); OUT.mkdir(exist_ok=True)
MAX_PER_PRODUCT = 1  # keep tiny for a smoke test

PRODUCTS = [
    "SWOT_L2_HR_RiverSP_2.0",
    "SWOT_L2_HR_LakeSP_2.0",
    "SWOT_L2_HR_Raster_2.0",
    "SWOT_L2_HR_PIXC_2.0",       # point cloud (big); single pass only
    "SWOT_L2_HR_PIXCVec_2.0",    # vectorized pixel cloud
]

ea.login()

# --------------- Helper: search + download a few ---------------
def fetch_sample(short_name, outdir: Path, n=1):
    res = ea.search_data(short_name=short_name,
                         temporal=(START, END),
                         bounding_box=OKAV_BBOX)
    if not res:
        print(f"[{short_name}] no granules found in window/bbox")
        return []
    # keep first n
    res = res[:n]
    tgt = outdir / short_name
    tgt.mkdir(parents=True, exist_ok=True)
    print(f"[{short_name}] downloading {len(res)} granule(s) to {tgt}")
    ea.download(res, tgt.as_posix())
    return [p for p in tgt.rglob("*") if p.is_file()]

# --------------- Helper: bbox GeoDataFrame for plotting ---------------
def bbox_gdf(bbox):
    minx, miny, maxx, maxy = bbox
    return gpd.GeoDataFrame(geometry=[gpd.GeoSeries.from_bbox((minx, miny, maxx, maxy)).geometry[0]],
                            crs="EPSG:4326")

B = bbox_gdf(OKAV_BBOX)

# --------------- RIVER / LAKE VECTORS (shapefile in folder or zip) ---------------
def read_first_vector(files):
    """Return a GeoDataFrame for the first .shp found (supports .zip)."""
    shp = None
    # try plain shapefile
    for f in files:
        if f.suffix.lower() == ".shp":
            shp = f
            break
    # try zipped shapefile
    if shp is None:
        for f in files:
            if f.suffix.lower() == ".zip":
                try:
                    # fiona can read 'zip://path.zip' directly
                    shp = f"zip://{f}"
                    gdf = gpd.read_file(shp)
                    return gdf.to_crs(4326)
                except Exception:
                    # fallback: unzip to temp dir
                    with zipfile.ZipFile(f) as z:
                        tmpdir = f.parent / (f.stem + "_unzipped")
                        tmpdir.mkdir(exist_ok=True)
                        z.extractall(tmpdir)
                        for p in tmpdir.rglob("*.shp"):
                            shp = p; break
                    if shp is None:
                        continue
                    break
    if shp is None:
        return None
    return gpd.read_file(shp).to_crs(4326)

def plot_vectors(gdf: gpd.GeoDataFrame, title: str):
    if gdf is None or gdf.empty:
        print(f"[{title}] nothing to plot")
        return
    ax = B.plot(edgecolor="k", facecolor="none", linewidth=1)
    gdf.clip(B.unary_union).plot(ax=ax, linewidth=0.7)
    ax.set_title(title)
    ax.set_xlabel("Lon"); ax.set_ylabel("Lat")
    plt.tight_layout(); plt.show()

# --------------- RASTER (NetCDF) ---------------
def read_first_nc(files):
    for f in files:
        if f.suffix.lower() == ".nc":
            return xr.open_dataset(f)
    return None

def pick_raster_var(ds: xr.Dataset):
    # prefer mask/height-like vars; otherwise first 2D var with lat/lon dims
    prefs = ["water_mask", "water_surface_height", "water_height", "water_area"]
    for v in prefs:
        if v in ds.data_vars:
            return v
    # generic 2D
    for v, dv in ds.data_vars.items():
        if {"lat","lon"}.issubset(set(dv.dims)) and dv.ndim == 2:
            return v
    # last resort
    return list(ds.data_vars)[0] if ds.data_vars else None

def plot_raster(ds: xr.Dataset, title: str, bbox=OKAV_BBOX):
    if ds is None:
        print(f"[{title}] no dataset")
        return
    # ensure lat/lon spatial dims
    latn = next((n for n in ds.coords if n.lower() in ("lat","latitude")), None)
    lonn = next((n for n in ds.coords if n.lower() in ("lon","longitude")), None)
    if latn is None or lonn is None:
        print(f"[{title}] missing lat/lon coords")
        return
    v = pick_raster_var(ds)
    if v is None:
        print(f"[{title}] no suitable variable found")
        return
    da = ds[v]
    # clip to bbox
    minx, miny, maxx, maxy = bbox
    sub = da.sel({lonn: slice(minx, maxx), latn: slice(miny, maxy)})
    # plot
    plt.figure(figsize=(7,5))
    plt.pcolormesh(sub[lonn], sub[latn], sub, shading="auto")
    plt.colorbar(label=v)
    plt.title(title)
    plt.xlabel("Lon"); plt.ylabel("Lat")
    plt.tight_layout(); plt.show()
    ds.close()

# --------------- PIXC / PIXCVec (point cloud / vectors) ---------------
def plot_pixc(ds: xr.Dataset, title: str, bbox=OKAV_BBOX, max_points=20_000):
    if ds is None:
        print(f"[{title}] no dataset")
        return
    # try common names
    lat = None; lon = None
    for cand in ("lat","latitude"):
        if cand in ds.variables: lat = cand; break
    for cand in ("lon","longitude"):
        if cand in ds.variables: lon = cand; break
    if lat is None or lon is None:
        print(f"[{title}] missing lat/lon vars")
        return
    latv = ds[lat].values.ravel()
    lonv = ds[lon].values.ravel()
    # bbox filter
    minx, miny, maxx, maxy = bbox
    m = (lonv>=minx)&(lonv<=maxx)&(latv>=miny)&(latv<=maxy)
    latv, lonv = latv[m], lonv[m]
    if latv.size == 0:
        print(f"[{title}] no points in bbox")
        return
    # thin for speed
    if latv.size > max_points:
        idx = np.linspace(0, latv.size-1, max_points).astype(int)
        latv, lonv = latv[idx], lonv[idx]
    plt.figure(figsize=(7,5))
    plt.scatter(lonv, latv, s=1, alpha=0.6)
    plt.title(title)
    plt.xlabel("Lon"); plt.ylabel("Lat")
    plt.tight_layout(); plt.show()
    ds.close()

# ---------------- Run: download & visualize ----------------
all_files = {}
for prod in PRODUCTS:
    files = fetch_sample(prod, OUT, n=MAX_PER_PRODUCT)
    all_files[prod] = files

# Rivers
plot_vectors(read_first_vector(all_files["SWOT_L2_HR_RiverSP_2.0"]), "SWOT RiverSP (vector) — Okavango")

# Lakes
plot_vectors(read_first_vector(all_files["SWOT_L2_HR_LakeSP_2.0"]), "SWOT LakeSP (vector) — Okavango")

# Raster
plot_raster(read_first_nc(all_files["SWOT_L2_HR_Raster_2.0"]), "SWOT Raster — Okavango", bbox=OKAV_BBOX)

# PIXC (point cloud; can be large – we only scatter a small sample)
plot_pixc(read_first_nc(all_files["SWOT_L2_HR_PIXC_2.0"]), "SWOT PIXC — Okavango sample", bbox=OKAV_BBOX)

# PIXCVec (vectorized pixel cloud) – often as shapefile/geojson; try to read like vectors
plot_vectors(read_first_vector(all_files["SWOT_L2_HR_PIXCVec_2.0"]), "SWOT PIXCVec (vector) — Okavango")


AttributeError: type object 'GeoSeries' has no attribute 'from_bbox'